# This Is The Teacher Reinforcement Learning Development Section

# Alur Fix ELM BertModel

In [1]:
import torch
import pickle
import numpy as np
import torch.nn.functional as F
from transformers import BertForMaskedLM, BertTokenizer
from torch.utils.data import DataLoader, Dataset

from utils import Utils
utils = Utils()

### Inisialisasi Target Untuk Testing

In [2]:
target = [[   6,  255,   77,    6,  140,    6,    6,    6,    6,    6,    6,    6,
            6,   77,   92,    6,    6,    6,    6,  135,    6,    6,  451,   11,
            6,    9, 2708,    6,    6,  353,    6, 3782],
        [  14,   25,   25,   92,   25,   92,  263,   85,  202,   92,  525,   79,
         1649,   25,  462,  469,   61, 1340,   92,  187,  165,  263,    6,   21,
         2303,   28, 2707,  219,   87,  167,  630, 3780],
        [  15,   34,  255,   25,    6,   25,  629,   86,  249,  283,  330,  283,
           25,    6,   70,   92,   25,   79,  283,  195,  299, 1117,  187,   22,
           25,  996,    5,   34,  103,  748,    6,   25],
        [  53,   99, 3622,   97,   99, 1428,    6,   87, 2016,   12,  235,  748,
          607,   92,  243,   34,  624,   25,  198,   17,  247, 1115,   99,  499,
          352,   28,   28,  117, 1850,   28, 2462,  579],
        [  21,  206,   39,    2,   12,   39,  519,    2,    6, 2383,   28,    6,
            2,  631,    6,    6,    2,  117,   28,    2,   66, 1118,    2,   21,
           70, 1244, 2110,   70, 3132,  932,  537,   39],
        [ 345,    2,    6,    0,   28,    6,    2,    0, 1535, 2366, 2078,   99,
            0,    6,   99,  206,    0,   12,  118,    0,    6,   70,    0,  332,
            6,   25,    2,   28,    2,    2,  446,   28],
        [ 671,    0, 1510,    0, 3811,   99,    0,    0,   21, 2379,    2,  206,
            0,  478,  206,   15,    0,  139,    2,    0,  202,  284,    0,   66,
           92, 2306,    0, 1154,    0,    0,   39,  589],
        [  21,    0,  429,    0, 3723,  336,    0,    0,    6,    5,    0,    2,
            0,   66,    2, 1499,    0,  209,    0,    0,  233,  596,    0, 3247,
            2,   28,    0,    2,    0,    0,   81,  397],
        [1296,    0,    2,    0,    2,   70,    0,    0, 1499,    6,    0,    0,
            0, 3870,    0,  615,    0,    2,    0,    0,    2,    2,    0,   39,
            0, 3660,    0,    0,    0,    0,  674,  653],
        [   9,    0,    0,    0,    0,    6,    0,    0,  249, 2158,    0,    0,
            0,    2,    0,  105,    0,    0,    0,    0,    0,    0,    0,  674,
            0,   21,    0,    0,    0,    0,    2,    2],
        [1142,    0,    0,    0,    0,  332,    0,    0,  238,    2,    0,    0,
            0,    0,    0, 2824,    0,    0,    0,    0,    0,    0,    0,    2,
            0,  141,    0,    0,    0,    0,    0,    0],
        [2530,    0,    0,    0,    0,    2,    0,    0,    6,    0,    0,    0,
            0,    0,    0,  512,    0,    0,    0,    0,    0,    0,    0,    0,
            0,   25,    0,    0,    0,    0,    0,    0],
        [   2,    0,    0,    0,    0,    0,    0,    0,  233,    0,    0,    0,
            0,    0,    0,    2,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    2,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    2,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]]

### Loading Bert model yang sudah di-fine tune

In [3]:
bert_ft_path = "Saved\\bert_finetuned_1000_data_fix.pt"

# Instantiate the model
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
# Assign the saved state dictionary to the model's state dictionary
model.load_state_dict(torch.load(bert_ft_path, map_location=torch.device('cpu')))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [4]:
i2b = "Saved\\msrvtt_index2bert_dic.p"
b2i = "Saved\\msrvtt_bert2index_dic.p"

with open(i2b, 'rb') as fp:
    index2bert = pickle.load(fp)
    
with open(b2i, 'rb') as fp:
    bert2index = pickle.load(fp)
    
targets = torch.tensor(target)
decoder_output = torch.rand(32, 5044)

index2bert_ts = torch.load('Saved\\index2bert_ts.pt').long()

In [5]:
(index2bert_ts.requires_grad_(False)).requires_grad

False

In [18]:
%%time

temperature = 3

bert_targets = index2bert_ts[targets.T].detach()

for t in range(1):
    # this iteration will be the decoding steps
    # and the input into the bert model
    inputs_id = bert_targets.detach()
    inputs_id[:, t][inputs_id[:, t] != 0] = 103
    
    with torch.no_grad():
        bert_output = model(inputs_id)[0]
        bert_output = torch.exp(F.log_softmax((bert_output / temperature), dim=-1))
    
    topk_bert_probs, topk_bert_indices = torch.topk(bert_output[:, t, :], k=50, dim=-1)
    topk_bert_indices[topk_bert_indices == 100] = 0 # untuk filter unknown word menjadi padding
    
    topk_indices = torch.full_like(topk_bert_indices, 3)
    output_tuple = (torch.eq(index2bert_ts.view(1, -1), topk_bert_indices.unsqueeze(-1))).nonzero(as_tuple=True)
    topk_indices[output_tuple[0], output_tuple[1]] = output_tuple[2]
    
    mask = torch.ones_like(topk_indices)
    # 3 adalah Unknown Token dan 0 adalah Padding Token
    mask[(topk_indices.eq(3) | topk_indices.eq(0))] = 0 
    
    kl_loss = utils.KLLoss(torch.gather(decoder_output, 1, topk_indices), 
                           topk_bert_probs,
                           mask, 
                           t)

Wall time: 11.8 s


In [11]:
topk_bert_probs[0]

tensor([0.0056, 0.0005, 0.0005, 0.0004, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003,
        0.0003, 0.0003, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002,
        0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002,
        0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002,
        0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002,
        0.0002, 0.0002, 0.0002, 0.0002, 0.0001])

In [14]:
torch.gather(decoder_output, 1, topk_indices)

tensor([[0.0504, 0.6130, 0.9854,  ..., 0.5427, 0.2433, 0.2433],
        [0.5275, 0.1613, 0.5181,  ..., 0.6169, 0.6169, 0.6169],
        [0.7475, 0.6516, 0.5980,  ..., 0.6445, 0.2859, 0.2859],
        ...,
        [0.3278, 0.6598, 0.6856,  ..., 0.0877, 0.0196, 0.1323],
        [0.7448, 0.3362, 0.4800,  ..., 0.8062, 0.6772, 0.0304],
        [0.2889, 0.3607, 0.0589,  ..., 0.3500, 0.3500, 0.4628]])

In [15]:
F.embedding(topk_indices, decoder_output)

IndexError: index out of range in self

# Optimization

In [ ]:
topk_indices = torch.tensor(np.frompyfunc(
                    lambda x: self.voc.bert2index.get(x, 3), 1, 1)(topk_bert_indices.detach().cpu().numpy()).astype('int32')).long().to(self.device)

In [91]:
# (topk_bert_indices == index2bert_ts.unsqueeze(-1)).nonzero(as_tuple=False)

topk_indices_ts = torch.full((32, 50), 3).long()
for i in range(32):
    indices = (torch.eq(index2bert_ts, topk_bert_indices[i].view(-1, 1))).nonzero(as_tuple=True)[1]
    topk_indices_ts[i][:indices.size(0)] = indices

In [110]:
topk_indices_ts = torch.full((32, 50), 3).long()
output_tuple = (torch.eq(index2bert_ts.unsqueeze(0), topk_bert_indices.unsqueeze(-1))).nonzero(as_tuple=True)
topk_indices_ts[output_tuple[0], output_tuple[1]] = output_tuple[2]

In [52]:
bert_indices[1][bert_indices[1] == 100].nonzero(as_tuple=True)

(tensor([0]),)

In [58]:
torch.where(bert_indices[1] == 100)

(tensor([5]), tensor([43]))

In [83]:
bert_indices[1][5]

tensor([ 2045,  2023,     0,  2009,  2002,  2182,  2008,  2016,  2029,  2054,
        12397,  8333,  2028,  2169,  2673,  2619,  2242,  3491,  1998, 12528,
        28137,  2166,  2073,  3793,  3256,  2059,  6662,  1610,  2678,  2178,
         2592,  2279, 21021, 18558, 17716,  2051,  2974, 12115, 25439,  2833,
         2061,  3967, 11247, 19769,  2600,  3941,  2946,  5789, 16967,  7284])

In [91]:
vocab_indices[0][vocab_indices[0]==2]

tensor([], dtype=torch.int64)

In [92]:
loss

[tensor(0.0407), tensor(0.0125), tensor(0.0064)]

In [44]:
print([index2word.get(tok_id) for tok_id in vocab_indices[1][0].numpy()])

['EOS', 'mixing', 'group', 'UNK', 'mixture', 'groups', 'band', 'consisting', 'mixes', 'composed', 'bands', 'UNK', 'mix', 'UNK', 'music', 'UNK', 'each', 'scores', 'UNK', 'together', 'UNK', 'UNK', 'UNK', 'pool', 'UNK', 'UNK', 'lots', 'UNK', 'pools', 'UNK', 'UNK', 'UNK', 'combining', 'UNK', 'UNK', 'UNK', 'footage', 'conference', 'UNK', 'UNK', 'mixed', 'UNK', 'simultaneously', 'UNK', 'dance', 'number', 'teams', 'UNK', 'UNK', 'lists']


In [45]:
print(tokenizer.convert_ids_to_tokens(bert_indices[1][0]))

['[PAD]', 'mixing', 'group', 'combination', 'mixture', 'groups', 'band', 'consisting', 'mixes', 'composed', 'bands', 'alliance', 'mix', 'blending', 'music', 'consists', 'each', 'scores', 'consist', 'together', 'consisted', 'majority', 'pairing', 'pool', 'confluence', 'association', 'lots', 'ensembles', 'pools', 'combinations', 'grouping', 'amalgamation', 'combining', 'composite', 'coalition', 'separation', 'footage', 'conference', 'equality', 'rivalry', 'mixed', 'celaena', 'simultaneously', 'choirs', 'dance', 'number', 'teams', 'presence', 'mingled', 'lists']


In [87]:
indices[:50]

tensor([   6,   28,  105,   21,  451,   39,  362,  140,  418,  162,  248,   36,
          66,  601,   59,   33,    5,   70,   81,  531,  177,  459,  144,   77,
         135, 1774,  139,  110,  328,  216,  154,   15,  469, 2831,  103,  295,
        3152,  430, 1269,   76,  355, 1603,  258,  990,  646,  460,  864,  179,
         776,  672])

In [76]:
tokenizer.convert_ids_to_tokens(topk_bert_indices[0]) == [index2word.get(tok_id) for tok_id in topk_indices_ts[0].detach().numpy()]

True

In [80]:
print([index2word.get(tok_id) for tok_id in topk_indices_ts[3].detach().numpy()])

['bowls', 'still', 'now', 'casually', 'he', 'television', 'profile', 'all', 'president', 'his', 'mark', 'tony', 'binoculars', 'while', 'chris', 'outdoors', 'fell', 'trek', 'hanging', 'these', 'pictures', 'PAD', 'EOS', 'tonight', 'looking', 'here', 'this', 'towards', 'slowly', 'you', '-', 'displaying', 'though', 'resting', 'mike', 'already', 'there', 'two', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK']


In [192]:
index2bert_ts[torch.where(index2bert_ts == topk_bert_indices[0].unsqueeze(-1))[1]]

tensor([1037, 1996, 2019, 1998, 2049, 1999, 2214, 2023, 2189, 2028, 2658, 2037,
        2007, 3080, 2235, 2304, 2006, 2000, 2010, 2047, 4854, 2312, 2048, 2045,
        2070, 2590, 2178, 2059, 2041, 2009, 2362, 1997, 2402, 2880, 2005, 2004,
        2021, 2367, 2204, 2195, 2502, 2026, 2008, 2122, 4748, 2119, 2307, 2013,
        2601, 3315])

In [191]:
topk_bert_indices[0]

tensor([1037, 1996, 2019, 1998, 2049, 1999, 2214, 2023, 2189, 2028, 2658, 2037,
        2007, 3080, 2235, 2304, 2006, 2000, 2010, 2047, 4854, 2312, 2048, 2045,
        2070, 2590, 2178, 2059, 2041, 2009, 2362, 1997, 2402, 2880, 2005, 2004,
        2021, 2367, 2204, 2195, 2502, 2026, 2008, 2122, 4748, 2119, 2307, 2013,
        2601, 3315])

In [139]:
# index di index2bert merupakan
# index yang ada di my custom vocabulary

# sedangkan value index2bert_ts adalah
# bert index

# inisialisasi dimana semua value adalah UNK Token
index2bert_ts = torch.full((5045,), 100).long()

# value diisi dengan token yang diketahui
for k, v in index2bert.items():
    index2bert_ts[k] = v

# indeks kedua diisi dengan padding
index2bert_ts[2] = 0
torch.save(index2bert_ts, 'Saved\\index2bert_ts.pt')

In [ ]:
for i in range(32):
    print(bert_targets_np[i] == bert_targets[i].numpy())

In [144]:
index2bert_ts = torch.load('Saved\\index2bert_ts.pt')

# CHECK

In [22]:
def KLLoss(decoder_output, bert_output, topk_indices, topk_bert_indices, t):
    UNK_TOKEN_ID = 3
    PAD_TOKEN_ID = 0
    mask = torch.ones_like(topk_indices)
    mask[(topk_indices.eq(UNK_TOKEN_ID) | topk_indices.eq(PAD_TOKEN_ID))] = 0
    
    log_loss_p = torch.log(torch.gather(decoder_output, 1, topk_indices))
    log_loss_p = log_loss_p * mask
    proba_q = torch.gather(bert_output[:, t], 1, topk_bert_indices)
    kl_loss = (-(proba_q * log_loss_p).sum(dim=-1)).mean()
    
    return kl_loss

In [24]:
KLLoss(decoder_output, bert_output_ls[1], vocab_indices[1], bert_indices[1], 1)

tensor(0.0140)

In [27]:
loss[2]

tensor(0.0129)

In [7]:
lambda_var = 0.3
mask_loss = torch.tensor([0.78879])
loss = 0
loss += (((1 - lambda_var) * mask_loss) + (lambda_var * kl_loss))

In [35]:
# path to pre-defined vocabulary file
vocab_file_path = "Saved\msrvtt_index2word_dic.p"
w2i = "Saved\msrvtt_word2index_dic.p"
i2w = "Saved\msrvtt_index2word_dic.p"

# load pre-defined vocabulary file
with open(vocab_file_path, "rb") as f:
    vocab = pickle.load(f)

with open(w2i, "rb") as f:
    word2index = pickle.load(f)  

with open(i2w, "rb") as f:
    index2word = pickle.load(f)

In [31]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
[index2word.get(token_id) for token_id in targets.T[1].detach().cpu().tolist()]

In [ ]:
[index2word.get(token_id) for token_id in vocab_indices[1][0].detach().cpu().tolist()]

## Fine-tuning BERT Model Using MSR-VTT Datasets

In [4]:
import torch
import pickle
from transformers import BertForMaskedLM, BertTokenizer

## Preparation video id yang digunakan

In [2]:
import json

path = "MSRVTT\\captions\\train_val_videodatainfo.json"

with open(path, 'r') as f:
    data = json.load(f)
    
train_id_list = [i for i in range(0, 850)]
val_id_list = [i for i in range(850, 900)]

## Membangun vocab untuk di-load Bert Tokenizer

In [5]:
# path to pre-defined vocabulary file
vocab_file_path = "Saved\msrvtt_index2word_dic.p"
w2i = "Saved\msrvtt_word2index_dic.p"
i2w = "Saved\msrvtt_index2word_dic.p"
my_vocab_path = "Saved\my_vocab.txt"

# load pre-defined vocabulary file
with open(vocab_file_path, "rb") as f:
    vocab = pickle.load(f)

with open(w2i, "rb") as f:
    word2index = pickle.load(f)  

with open(i2w, "rb") as f:
    index2word = pickle.load(f)
    
# with open(my_vocab_path, 'w') as file:
#     for _, v in vocab.items():
#         file.write(v + '\n')

## Inisialisasi Bert Tokenizer menggunakan metode vocab yang sama dengan metode video cpationing

In [13]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

my_vocab_path = "Saved\my_vocab.txt"
my_tokenizer = BertTokenizer(vocab_file=my_vocab_path, pad_token="PAD", unk_token="UNK")
# len(my_tokenizer.vocab)

## Memotong Vocab yang Digunakan

In [14]:
# get the vocab and store it in list type
tokens = list(tokenizer.vocab.keys())
my_vocab = list(vocab.values())

In [15]:
indexes = [0, 1, 2, 3]
values = ['[PAD]', '[CLS]', '[SEP]', '[UNK]']

for (index, replacement) in zip(indexes, values):
    my_vocab[index] = replacement

my_vocab.append("[MASK]")

In [16]:
indices = [tokenizer.vocab[token] for token in my_vocab if token in tokens]
my_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', vocab_size=len(indices), unk_token='[UNK]')
my_tokenizer.vocab = {k: v for k, v in tokenizer.vocab.items() if v in indices}

In [131]:
my_tokenizer.vocab_size

4335

## Menghubungkan Token IDs Methode Tokenizer CAP dan BERT

In [17]:
index2bert = {}
bert2index = {}

for i, token in enumerate(my_vocab):
    if token in tokens:
        index2bert[i] = tokenizer.vocab[token]
        bert2index[tokenizer.vocab[token]] = i

In [18]:
i2b = "Saved\\index2bert_dic.p"
b2i = "Saved\\bert2index_dic.p"

with open(i2b, 'wb') as fp:
    pickle.dump(index2bert, fp, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(b2i, 'wb') as fp:
    pickle.dump(bert2index, fp, protocol=pickle.HIGHEST_PROTOCOL)

## Melakukan Encode

In [50]:
def encode_caption(data, id_list): 
    encoded_captions = []

    for caption in data['sentences']:
        if int(caption['video_id'][5:]) in id_list:
            encoded_caption = tokenizer.encode_plus(
                caption['caption'],
                padding='max_length',
                max_length=128,
                return_tensors='pt'
            )
            encoded_captions.append(encoded_caption)
        
    return encoded_captions

In [51]:
encoded_captions = encode_caption(data, train_id_list)
val_encoded_captions = encode_caption(data, val_id_list)

torch.save(encoded_captions, 'MSRVTT\\captions\\encoded_captions_for_bert.pt')
torch.save(val_encoded_captions, 'MSRVTT\\captions\\val_encoded_captions_for_bert.pt')

In [61]:
encoded_path = 'MSRVTT\\captions\\encoded_captions_for_bert.pt'
val_encoded_path = 'MSRVTT\\captions\\val_encoded_captions_for_bert.pt'

encoded_captions = torch.load(encoded_path)
vaval_encoded_captions = torch.load(val_encoded_path)


## Preparing For Training

In [2]:
import torch
from transformers import AdamW
from torch.utils.data import DataLoader, Dataset

In [39]:
train_loader = DataLoader(encoded_captions, batch_size=32, shuffle=True)
val_loader = DataLoader(val_encoded_captions, batch_size=32)

In [ ]:
# Define the fine-tuned BERT model
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.train()

# Define the optimizer and learning rate
optimizer = AdamW(model.parameters(), lr=3e-5)

# Define the loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Train the model on the caption data
for epoch in range(10):
    for data in train_loader:
        input_ids = data['input_ids'].squeeze(1)
        attention_mask = data['attention_mask'].squeeze(1)
        
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    
    # Evaluate the model on the validation set
    with torch.no_grad():
        val_loss = 0.0
        for data in val_loader:
            input_ids = data['input_ids'].squeeze(1)
            attention_mask = data['attention_mask'].squeeze(1)
        
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
            val_loss += outputs.loss.item() * input_ids.shape[0]
        val_loss /= len(val_dataset)
    
    print(f'Epoch {epoch+1}: Train Loss = {loss.item():.3f}, Val Loss = {val_loss:.3f}')

In [54]:
for data in train_loader:
    print(data['attention_mask'].squeeze(1).shape)
    break

torch.Size([128, 128])


# Training Algorithm For Base-TRL

In [2]:
target = [[   6,  255,   77,    6,  140,    6,    6,    6,    6,    6,    6,    6,
            6,   77,   92,    6,    6,    6,    6,  135,    6,    6,  451,   11,
            6,    9, 2708,    6,    6,  353,    6, 3782],
        [  14,   25,   25,   92,   25,   92,  263,   85,  202,   92,  525,   79,
         1649,   25,  462,  469,   61, 1340,   92,  187,  165,  263,    6,   21,
         2303,   28, 2707,  219,   87,  167,  630, 3780],
        [  15,   34,  255,   25,    6,   25,  629,   86,  249,  283,  330,  283,
           25,    6,   70,   92,   25,   79,  283,  195,  299, 1117,  187,   22,
           25,  996,    5,   34,  103,  748,    6,   25],
        [  53,   99, 3622,   97,   99, 1428,    6,   87, 2016,   12,  235,  748,
          607,   92,  243,   34,  624,   25,  198,   17,  247, 1115,   99,  499,
          352,   28,   28,  117, 1850,   28, 2462,  579],
        [  21,  206,   39,    2,   12,   39,  519,    2,    6, 2383,   28,    6,
            2,  631,    6,    6,    2,  117,   28,    2,   66, 1118,    2,   21,
           70, 1244, 2110,   70, 3132,  932,  537,   39],
        [ 345,    2,    6,    0,   28,    6,    2,    0, 1535, 2366, 2078,   99,
            0,    6,   99,  206,    0,   12,  118,    0,    6,   70,    0,  332,
            6,   25,    2,   28,    2,    2,  446,   28],
        [ 671,    0, 1510,    0, 3811,   99,    0,    0,   21, 2379,    2,  206,
            0,  478,  206,   15,    0,  139,    2,    0,  202,  284,    0,   66,
           92, 2306,    0, 1154,    0,    0,   39,  589],
        [  21,    0,  429,    0, 3723,  336,    0,    0,    6,    5,    0,    2,
            0,   66,    2, 1499,    0,  209,    0,    0,  233,  596,    0, 3247,
            2,   28,    0,    2,    0,    0,   81,  397],
        [1296,    0,    2,    0,    2,   70,    0,    0, 1499,    6,    0,    0,
            0, 3870,    0,  615,    0,    2,    0,    0,    2,    2,    0,   39,
            0, 3660,    0,    0,    0,    0,  674,  653],
        [   9,    0,    0,    0,    0,    6,    0,    0,  249, 2158,    0,    0,
            0,    2,    0,  105,    0,    0,    0,    0,    0,    0,    0,  674,
            0,   21,    0,    0,    0,    0,    2,    2],
        [1142,    0,    0,    0,    0,  332,    0,    0,  238,    2,    0,    0,
            0,    0,    0, 2824,    0,    0,    0,    0,    0,    0,    0,    2,
            0,  141,    0,    0,    0,    0,    0,    0],
        [2530,    0,    0,    0,    0,    2,    0,    0,    6,    0,    0,    0,
            0,    0,    0,  512,    0,    0,    0,    0,    0,    0,    0,    0,
            0,   25,    0,    0,    0,    0,    0,    0],
        [   2,    0,    0,    0,    0,    0,    0,    0,  233,    0,    0,    0,
            0,    0,    0,    2,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    2,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    2,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]]

## Loss Cross Entropy

In [1]:
import torch
import torch.nn.functional as F
import numpy as np

In [3]:
bert_ft_path = "Saved\\bert_finetuned_1000_data_fix.pt"

saved_state = torch.load(bert_ft_path, map_location=torch.device('cpu'))
# Instantiate the model
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
# Assign the saved state dictionary to the model's state dictionary
model.load_state_dict(saved_state)

# my_vocab_path = "Saved\my_vocab.txt"

NameError: name 'BertForMaskedLM' is not defined

In [5]:
targets = torch.tensor(target)

In [9]:
i2b = "Saved\\index2bert_dic.p"
b2i = "Saved\\bert2index_dic.p"

with open(i2b, 'rb') as fp:
    index2bert = pickle.load(fp)
    
with open(b2i, 'rb') as fp:
    bert2index = pickle.load(fp)

In [45]:
t = targets[2].view(-1, 1)

In [307]:
target_ts = torch.tensor(target)

In [314]:
target_ts.T.shape

torch.Size([32, 14])

In [146]:
def get_bert_index(t):
    return torch.tensor([index2bert.get(dic_idx, 100) for dic_idx in t.squeeze(1).cpu().tolist()]).long()

def get_token_index(bert_indices, bert2index):
    return torch.tensor([[bert2index.get(bert_id, 3) for bert_id in bert_indices[i].cpu().tolist()] for i in range(bert_indices.size(0))]).long()

In [148]:
bert_t = get_bert_index(t)
bert_t.shape

torch.Size([32])

In [177]:
max_length = 28

for t in range(max_length):
    print(targets[t])
    break

tensor([   6,  255,   77,    6,  140,    6,    6,    6,    6,    6,    6,    6,
           6,   77,   92,    6,    6,    6,    6,  135,    6,    6,  451,   11,
           6,    9, 2708,    6,    6,  353,    6, 3782])


In [255]:
target_variable = targets.T
for t in range(targets.size(0)):
    if t != 0:
        for j in range(targets.size(1)):
            if target_variable[j][t] != 0:
                target_variable[j][t] = 103
        break

In [276]:
bert_t = get_bert_index(targets[0].view(-1, 1))

## My Algorithm Code

In [451]:
bert_inputs = targets.T.detach().cpu().tolist()

## Minot Improvement Code

In [460]:
bert_targets = torch.tensor([[index2bert.get(token_id.item(), 100) for token_id in batch] for batch in targets.T])

## There is issue in using np.vectorize

In [467]:
# change the '32' number to cfg.batch_size for the later implementation
bert_targets_np = np.hstack([np.array([[101]] * 32), np.vectorize(index2bert.get)(targets.T.detach().cpu().numpy(), 100)])
bert_targets = torch.tensor(bert_targets_np)

## Try using lambda ufunc

In [471]:
bert_targets_np = np.hstack([np.array([[101]] * 32), np.frompyfunc(lambda x: index2bert.get(x, 100), 1, 1)(targets.T.detach().cpu().numpy())]).astype(np.int64)
bert_targets = torch.tensor(bert_targets_np).long()

In [ ]:
#  '[PAD]': 0,
#  '[UNK]': 100,
#  '[CLS]': 101,
#  '[SEP]': 102,
#  '[MASK]': 103,

In [347]:
[101] + bert_inputs[0]

[101, 6, 14, 15, 53, 21, 345, 671, 21, 1296, 9, 1142, 2530, 2, 0]

In [359]:
new_bert_inputs = []

for i in range(targets.T.size(0)):
    temp = [101]
    for j in range(targets.T.size(1)):
        temp.append(index2bert.get(bert_inputs[i][j]))
    
    new_bert_inputs.append(temp)

## Function for preparing Bert Inputs

Jadi fungsi ini digunakan untuk merubah token IDs dari target variable menjadi Bert Token IDs serta menambahkan [CLS] token di awal sentence. Hal ini juga dilakukan agar inference Bert Optimal dimana Bert perlu menerima seluruh kalimat dan menambah token [MASK] pada posisi tertentu.

In [7]:
import numpy as np

In [429]:
bert_inputs_ls = [[101] + [index2bert.get(token_id, 100) for token_id in batch] for batch in bert_inputs]

In [208]:
max_target_len = targets.size(0)

bert_targets_np = np.frompyfunc(
        lambda x: index2bert.get(x, 100) if x != 2 else index2bert.get(0, 0),
        1, 1
    )(targets.T.detach().cpu().numpy()).astype('int32')

for t in range(3):
    # this iteration will be the decoding steps
    # and the input into the bert model
    inputs_id = torch.tensor(bert_targets_np).long()
    inputs_id[:, t+1][inputs_id[:, t+1] != 0] = 103
    print(inputs_id[0])

tensor([ 1037,   103,  1997,  2273,  1998,  2308, 10955,  1998, 11278,  2012,
         2712,  5370,     0,     0])
tensor([ 1037,  2177,   103,  2273,  1998,  2308, 10955,  1998, 11278,  2012,
         2712,  5370,     0,     0])
tensor([ 1037,  2177,  1997,   103,  1998,  2308, 10955,  1998, 11278,  2012,
         2712,  5370,     0,     0])


In [ ]:
my_tokenizer.convert_ids_to_tokens(inputs_id[0])

In [236]:
decoder_output = torch.rand(32, 5044)
# decoder_output = decoder_output.unsqueeze(0)

In [237]:
topk_indices.shape

(32, 50)

In [256]:
outputs[0][:, t+1].shape

torch.Size([32, 30522])

In [282]:
q = torch.tensor([[0.3, 0.2, 0.1], 
                  [0.3, 0.2, 0.1]])
print(q)
p = torch.tensor([[0.1, 0.2, 0.3], 
                  [0.3, 0.2, 0.1]])
print(p.T)
torch.matmul(q, p.T)

tensor([[0.3000, 0.2000, 0.1000],
        [0.3000, 0.2000, 0.1000]])
tensor([[0.1000, 0.3000],
        [0.2000, 0.2000],
        [0.3000, 0.1000]])


tensor([[0.1000, 0.1400],
        [0.1000, 0.1400]])

In [265]:
temperature = 3

with torch.no_grad():
    outputs = model(inputs_id)[0]
    outputs = F.softmax((outputs / temperature), dim=-1)

topk_bert_indices = torch.argsort(outputs, dim=-1, descending=True)[:, t+1, :50] # ambil predicted word pada timestep t
# proba_q = torch.gather(outputs[:, t+1], 1, topk_bert_indices)

topk_indices = torch.tensor(np.frompyfunc(
        lambda x: bert2index.get(x, 3),1, 1)(topk_bert_indices.detach().cpu().numpy()).astype('int32')).long()

In [318]:
from utils import Utils

utils = Utils()

In [319]:
kl_loss = utils.KLLoss(decoder_output, outputs, topk_indices, topk_bert_indices, t=3)
print(kl_loss)

TypeError: KLLoss() got an unexpected keyword argument 't'

In [299]:
def KLLoss(decoder_output, bert_output, topk_indices, topk_bert_indices):
    log_loss_p = torch.log(torch.gather(decoder_output, 1, topk_indices))
    proba_q = torch.gather(outputs[:, t+1], 1, topk_bert_indices)
    kl_loss = (-(proba_q * log_loss_p).sum(dim=-1)).mean()
    return kl_loss

In [302]:
kl_loss = KLLoss(decoder_output, outputs, topk_indices, topk_bert_indices)
kl_loss

tensor(0.0180)

In [163]:
predictions = outputs[0][0, 7].topk(k=20).indices.tolist()

predicted_tokens = [my_tokenizer.decode(token_id) for token_id in predictions]
print(predicted_tokens)

['a n d', 't e a', 't o', 'o r', 't r e y', 'w i t h', 'a', 'a c', 't a m', 'a l l', 's e v e r a l', '...', 'n o', 't h e n', 'a t', 'a l', '[ S E P ]', 's h e', 'n o r', '[ C L S ]']


In [202]:
input_sequence = "a woman with [MASK] teaching how to trim flowers"

# Tokenize the input sequence and get the index of the masked token
input_tokens = my_tokenizer.tokenize(input_sequence)
mask_index = input_tokens.index("[MASK]")
input_ids = my_tokenizer.convert_tokens_to_ids(input_tokens)

# Replace the masked token with its corresponding ID
input_ids[mask_index] = tokenizer.mask_token_id

# Convert input IDs to PyTorch tensor and add a batch dimension
input_tensor = torch.tensor([input_ids])

# Predict the masked word
with torch.no_grad():
    output = model(input_tensor)

predicted_token_id = torch.argmax(output[0][0][mask_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens(predicted_token_id)

print(f"Predicted token: {predicted_token}")

Predicted token: bingham


In [203]:
predictions = output[0][0, mask_index].topk(k=50).indices.tolist()

predicted_tokens = [my_tokenizer.decode(token_id) for token_id in predictions]
print(predicted_tokens)

['b i n g h a m', 'r e i d', 'p r o', 'n a', 'i o n a', 'a d d i t i o n', 'p c', 'p e r h a p s', 'b r u c e', 'c l i n t o n', 'e f f e c t i v e l y', 'w o r k', 'f u r t h e r', 'a l s o', 'c o n g r e s s', 'g r o w t h', 'f r a s e r', 'b n', 'b y r n e', 'i m p a c t', 'f r', 'e a r l y', 'c o n t i n u e d', 'i n p u t', 'g r o u p', 'e d', 'a', 'h o l m e s', 's h o c k', 'h i m s e l f', 'c r e d i t', 'b o o k', 'a n d', 'c o r n e l l', 'd a v i d', 'l a t e r', 'f i r s t', 'e x p e r i e n c e', 'c o a', 'r e s e a r c h', 'e v i d e n c e', 'f i l i n g', 'n o', 'm a x', 'g r o w i n g', 's e r v i c e', 't o', 'c o n t i n u i n g', 't h i s', 'c b c']


## Suggestion

In [444]:
max_target_len = targets.size(0)

input_ids = torch.tensor([[101] + [index2bert.get(token_id, 100) for token_id in batch] for batch in bert_inputs])

for t in range(max_target_len):
    # this iteration will be the decoding steps
    # and the input into the bert model
    if t < 3:
        input_ids[:, t].masked_fill_(input_ids[:, t] != 0, 103)
        print(input_ids[0])
    else:
        break

tensor([  103,  1037,  2177,  1997,  2273,  1998,  2308, 10955,  1998, 11278,
         2012,  2712,  5370,   102,     0])
tensor([  103,   103,  2177,  1997,  2273,  1998,  2308, 10955,  1998, 11278,
         2012,  2712,  5370,   102,     0])
tensor([  103,   103,   103,  1997,  2273,  1998,  2308, 10955,  1998, 11278,
         2012,  2712,  5370,   102,     0])


In [478]:
bert_targets_np[0]

array([  101,  1037,  2177,  1997,  2273,  1998,  2308, 10955,  1998,
       11278,  2012,  2712,  5370,   102,     0], dtype=int64)

In [ ]:
[my_tokenizer.convert_ids_to_tokens(bert_token) for bert_token in bert_inputs[0]]

In [119]:
sentence = "I want to eat [MASK] for lunch today."

tokenized_text = my_tokenizer.tokenize(sentence)

In [85]:
masked_index = len(tokenized_text) - 1
tokenized_text[masked_index] = '[MASK]'

In [121]:
# Convert tokens to IDs
indexed_tokens = my_tokenizer.convert_tokens_to_ids(tokenized_text)

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])

with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0][0][4].topk(k=20).indices.tolist()
    predicted_tokens = my_tokenizer.convert_ids_to_tokens(predictions)
    print(predicted_tokens)

['[PAD]', '[CLS]', 'backlash', 'mace', 'effectively', '##fare', 'excess', 'thanksgiving', 'class', 'multiple', 'alcohol', 'dish', 'chef', 'conflict', 'vlad', 'food', 'jared', 'chefs', 'violence', 'host']


## Inference BertForMaskedLM Example

In [212]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [217]:
my_tokenizer.add_tokens('[MASK]')

1

In [228]:
sentence = "a woman with green [MASK]"

# Tokenize input sentence
tokens = my_tokenizer.tokenize(sentence)
mask_idx = tokens.index('[MASK]')
input_ids = torch.tensor([tokenizer.encode(sentence, add_special_tokens=True)])

In [326]:
input_ids

tensor([[ 101, 1037, 2450, 2007, 2665,  103,  102]])

In [ ]:
with torch.no_grad():
    outputs = model(input_ids)
    predictions = outputs[0][0, mask_idx].topk(k=50).indices.tolist()

predicted_tokens = [tokenizer.decode(token_id) for token_id in predictions]
print(predicted_tokens)

In [249]:
outputs[0].shape

torch.Size([1, 7, 30522])

Jadi index [0] pertama itu untuk mengambil tensor logits. Lalu [0] kedua adalah untuk mengambil batch, karena batchnya hanya satu untuk saat ini. Lalu mask_idx adalah dimensi indeks token yang diambil.

In [246]:
outputs[0][0, mask_idx]

tensor([-2.2777, -7.6340, -7.9684,  ..., -7.5812, -5.0998, -8.6828])

## KL Loss

In [206]:
output[0].shape

torch.Size([1, 9, 30522])

## Miscelaneous

In [ ]:
[index2word.get(idx) for idx in targets[0].cpu().tolist()]

In [153]:
with torch.no_grad():
    logits = model(bert_t.view(-1, 1))
    
bert_indices = torch.argsort(logits[0].squeeze(1), dim=1)[:, :50].long()

In [165]:
bert_indices

tensor([[17007, 27004, 28878,  ..., 26331, 27690, 17181],
        [24229, 22968, 28397,  ..., 17766, 11656, 29504],
        [23695, 28666, 18926,  ..., 25393, 11300, 27388],
        ...,
        [25495, 13900, 17661,  ..., 25027, 15994, 21322],
        [22925, 29553, 23630,  ..., 19859, 19948, 21273],
        [27920, 18926, 14918,  ..., 12335, 27245, 17007]])

In [102]:
batch_size = 32
num_words = 50

default_value = torch.full((batch_size, num_words), 3, dtype=torch.long)
bert_indices = bert_indices.long()
soft_predict_index = torch.gather(default_value, 1, bert2index.get(bert_indices, default_value))

In [104]:
soft_predict_index.shape

torch.Size([32, 50])

In [94]:
decoder_output = torch.rand(32, 5044)

In [52]:
bert_indices.shape

torch.Size([32, 50])

## Miscellaneous

In [ ]:
tokenizer.convert_ids_to_tokens(bert_t)

In [212]:
result = model(bert_t)

In [213]:
indices = torch.argsort(result[0].squeeze(1), dim=-1, descending=True)[:, :50]

In [214]:
max_indices = torch.argmax(result[0].squeeze(1), dim=-1)
max_indices

tensor([   0,    0,  102,    0,  102,    0,    0,    0,    0,    0, 1998,    0,
           0,  102,    0,    0,    0,    0,    0, 4183,  102,  101,  101,    0,
           0,    0,    0,    0,    0,    0,  102,    0])

In [ ]:
tokenizer.convert_ids_to_tokens(indices[19])